In [10]:
import numpy as np
import pandas as pd

data=pd.read_csv('onlyhash.csv',header=None, sep='\t')
data = data.rename(columns={0: "userid", 1: "date",2: "hashtag"})

def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

data=explode(data.assign(hashtag=data.hashtag.str.split(' ')), 'hashtag')
grp1 = data.groupby('date')
#print(grp1.head())
grp1.ngroups
dict1 = grp1.groups
dict1
dict_htags = {}

for key in dict1:
    list_htags = []
    for i in range(len(dict1[key])):
        list_htags.append(data.loc[[dict1[key][i]]])
    dict_htags[key]  = list_htags
    
#dictionary with date as key and hashtags used on each date as values
print(dict_htags)


KeyboardInterrupt: 